In [29]:
import os
import pandas as pd
BATCHES_FOLDER = "MTurkBatchesResults"
OUTPUT_FILEMANE = "output.csv"

This code finds the "truth" - annotators agreements and stores to file

In [30]:
mturk_batches = ["{}/{}".format(BATCHES_FOLDER, fl) for fl in os.listdir(BATCHES_FOLDER) if fl.endswith(".csv")]
print(mturk_batches)
#modified by hand
truth_df = pd.read_csv("{}/{}".format(BATCHES_FOLDER, "truth.csv"))
# mturk_batches += ["{}/{}".format(BATCHES_FOLDER, "truth.csv") for i in range(5)]

['MTurkBatchesResults/Batch_3222045_batch_results.csv', 'MTurkBatchesResults/Batch_3227135_batch_results (1).csv', 'MTurkBatchesResults/truth.csv', 'MTurkBatchesResults/Batch_3222021_batch_results.csv', 'MTurkBatchesResults/Batch_3227073_batch_results.csv', 'MTurkBatchesResults/Batch_3227115_batch_results.csv']


In [31]:
truth_df.head(10)

,Input.company,Input.url,Answer.ClimateConfidence,Answer.CompanyConfidence,Answer.sentiment,Input.content,Input.extract,Input.title,comments
0,Goodyear Tire & Rubber Co,http://www.performancetopeka.com/shop-for-tire...,0.0,2.0,NaN,"<HTML lang=""en-US"">\n<style type=""text/css"">\n...",The <b>Goodyear Tire &amp; Rubber Company</b> ...,"Shop Goodyear Tires in Topeka, KS :: Performan...",d
1,BASF AG,https://www.euractiv.com/section/trade-society...,2.0,0.0,NaN,"<HTML class=""no-js"" lang=""en-GB"" moznomarginbo...",... “It depends on what the public is willing ...,EU's re-industrialisation dream 'hostage' of h...,d
2,Bridgestone Corp.,https://www.bcorporation.net/blog/climate-change,2.0,0.0,NaN,"<HTML class=""bcorp"" lang=""en"" dir=""ltr"" prefix...","<b>Climate</b> Ride, B <b>Corps</b>, and Colle...",Climate change | B Corporation,d
3,General Motors Corp.,http://www.greengrowthknowledge.org/blog/why-g...,1.0,2.0,Positive,"<HTML class=""no-js not-oldie"" lang=""en"" dir=""l...",... In November 2012 <b>General Motors</b> (<b...,Why General Motors is developing new electric ...,d
4,Ford Motor Co.,https://www.climaterealityproject.org/ford,1.0,2.0,Positive,"<HTML lang=""en"" dir=""ltr"" prefix=""content: htt...",Tell Bill Ford and <b>Ford Motor Company</b>: ...,Tell Bill Ford and Ford Motor Company: No U-Tu...,d
5,Exxon Mobil Corp.,http://fortune.com/2017/03/29/exxon-mobil-dona...,2.0,2.0,Negative,"<HTML lang=""en"">\n<style type=""text/css"">\nA:b...","... <b>ExxonMobil</b>, the world&#39;s largest...",Exxon Mobil Wants Donald Trump to Keep the Par...,d
6,Nestle SA,http://www.sustainablebrands.com/news_and_view...,2.0,1.0,Positive,"<HTML xmlns=""http://www.w3.org/1999/xhtml"" xml...",... To make the A-List of <b>Climate</b> Polic...,"Report: Unilever, Tesla, Nestlé Among Biggest ...",d
7,Toyota Motor Corp.,https://www.greenbuildermedia.com/green-builde...,2.0,1.0,Positive,"<HTML class=""no-js"" lang=""en"">\n<style type=""t...",<b>Toyota Motor Corp</b>. Here&#39;s a six-pro...,Green Builder Media's 2017 Eco-Leaders,NaN
8,Duke Energy,https://www.facingsouth.org/2016/02/nc-environ...,1.0,2.0,Positive,"<HTML lang=""en"" dir=""ltr"" prefix=""og: http://o...",... <b>Duke Energy</b> officials in recent wee...,NC environmental officials: Too anti-regulatio...,d?
9,BASF AG,https://synbiobeta.com/commercializing-synthet...,2.0,1.0,Neutral,"<HTML class="""" lang=""en-US"" prefix=""og: http:/...",... Innovation to Tackle <b>Climate Change</b>...,Innovation to Tackle Climate Change and Feed a...,d


In [32]:
import numpy as np
df = pd.concat([pd.read_csv(batch) for batch in mturk_batches])
# df = df[df['Answer.sentiment'].notnull()]

df = df.dropna(subset = ['Answer.CompanyConfidence'])
truth_df = truth_df.dropna(subset = ['Answer.CompanyConfidence'])
# df = df[np.isfinite(df['Answer.sentiment'])]
urls = df.groupby("Input.url", as_index=False)

In [33]:
# pd.options.display.max_colwidth = 10000
# row = df.loc[df['Input.url'] == 'http://bellona.org/news/arctic/2017-04-trump-like-friend-of-putin-tells-him-climate-change-isnt-man-made']
# row['Input.company']
# df.columns
len(df)

462

In [34]:
columns = ['url', 'company', 'short_text', 'title', 'full_text', 'Answer.ClimateConfidence', 'Answer.CompanyConfidence', 'Answer.sentiment']
result_df = pd.DataFrame(columns=columns)
# agree_level = []
cf = []

for num, url in enumerate(list(urls.groups.keys())):
    results = df.loc[df['Input.url'] == url]
#     if 'w.ohio.com/akron/writers/katie-byard/goodyear-bridgestone-team-up-to-distribute-tires-nationallly-new-tirehub-will-be-one-of-the-largest-such-joint-ventures-in-the-country' in url:
#         import ipdb; ipdb.set_trace()
    key = results['Input.extract'].keys()[0]
    vals = [url, results['Input.company'][key], results['Input.extract'][key], results['Input.title'][key], results['Input.content'][key]]
    for column_name in ['Answer.ClimateConfidence', 'Answer.CompanyConfidence', 'Answer.sentiment']:
        #agree level == 2

        if any(truth_df['Input.url'] == url) and truth_df.loc[truth_df['Input.url'] == url][column_name].iloc[0] != np.nan:
            vals.append(truth_df.loc[truth_df['Input.url'] == url][column_name].iloc[0])
#             if column_name =='Answer.CompanyConfidence' and np.isnan(truth_df.loc[truth_df['Input.url'] == url][column_name].iloc[0]):
#                 import ipdb; ipdb.set_trace()
#                 cf.append(truth_df.loc[truth_df['Input.url'] == url][column_name].iloc[0])
            continue
        
        counter = results[column_name].value_counts()
        if len(counter) > 0:
            highest = max(counter.values)
            #         agree_level.append("{} of {}".format(highest, sum(counter.values)))
            res = [k for k, v in counter.items() if v == highest]
            vals.append(res[0])
            continue
        vals.append(None)
    result_df.loc[num] = vals
# result_df['agree_level'] = agree_level

In [35]:
len(result_df)

348

In [36]:
result_df['company'].unique()

# df.loc[result_df['Штзгею'] == result_df['Answer.CompanyConfidence'].unique()[-1]]
# dr

array(['GazProm', 'RosNeft', 'Chevron', 'Petro China',
       'Royal Dutch Shell PLC', 'BMW', 'Goodyear Tire & Rubber Co',
       'Toyota Motor Corp.', 'Exxon Mobil Corp.', 'Cargill Inc', 'Ineos',
       'BASF AG', 'EDS Corp.', 'ABB', 'Petrobras',
       'Sumitomo Rubber Industries', 'Amazon.com Inc.', 'Ford Motor Co.',
       'Volkswagen AG', 'Daimler AG', 'General Motors Corp.', 'BASF',
       'Apple Inc', 'Bayer AG', 'Michelin', 'PepsiCo Inc.', 'Nestle SA',
       'Johnson & Johnson', 'IBM', 'Bridgestone Corp.', 'Duke Energy',
       'BP plc', 'Chevron Corp.', 'Dow Chemical'], dtype=object)

In [38]:
result_df.to_csv(OUTPUT_FILEMANE, index=False)